# Day 35

Today I'm going to pull ADP data from as many seasons as possible. I'll utilize the public API from [Fantasy Football Calculator](https://help.fantasyfootballcalculator.com/article/42-adp-rest-api).  

Eventually I want to see how a player's fantasy football production compares to their ADP. Is it the case that the top drafted players actually end up performing the best? How does this comparison hold up historically?

In [30]:
import pandas as pd
import numpy as np
import requests
import sqlite3

# Create database connection
conn = sqlite3.connect('../../data/db/database.db')

In [25]:
def fantasy_calculator_adp(scoring='ppr', teams=12, season=2022):
    
    try:
        base_url = 'https://fantasyfootballcalculator.com'
        
        response = requests.get(f"{base_url}/api/v1/adp/{scoring}?teams={teams}&year={season}")
        
        df = pd.DataFrame(response.json()['players'])

        # Add identification columns
        df['season'] = season
        df['teams'] = teams
        df['scoring'] = scoring

        return df
    
    except:
        print(f"No data available for combination: {scoring}, {teams}, {season}")

In [27]:
df = fantasy_calculator_adp(scoring='ppr', teams=12, season=2021)
df.head()

,player_id,name,position,team,adp,adp_formatted,times_drafted,high,low,stdev,bye,season,teams,scoring
0,2434,Christian McCaffrey,RB,CAR,1.2,1.01,1368,1,5,0.5,13,2021,12,ppr
1,2432,Dalvin Cook,RB,MIN,2.5,1.03,1292,1,6,0.7,7,2021,12,ppr
2,2439,Alvin Kamara,RB,NO,3.8,1.04,1500,1,8,1.0,14,2021,12,ppr
3,2350,Derrick Henry,RB,TEN,4.0,1.04,460,1,8,1.3,6,2021,12,ppr
4,2343,Ezekiel Elliott,RB,DAL,5.3,1.05,1148,2,12,1.1,9,2021,12,ppr


In [40]:
seasons = list(np.arange(2007, 2022))
teams = [10, 12]
scoring_systems = ['standard', 'half-ppr', 'ppr']

dfs = []

for scoring_system in scoring_systems:
    # print(f"Working on scoring system: {scoring_system}")
    for team in teams:
        # print(f"Working on team: {team}")
        for season in seasons:
            # print(f"Working on season: {season}")
            dfs.append(fantasy_calculator_adp(scoring_system, team, season))

df_all = pd.concat(dfs)
df_all


No data available for combination: half-ppr, 10, 2007
No data available for combination: half-ppr, 10, 2008
No data available for combination: half-ppr, 10, 2009
No data available for combination: half-ppr, 10, 2010
No data available for combination: half-ppr, 10, 2011
No data available for combination: half-ppr, 10, 2012
No data available for combination: half-ppr, 10, 2013
No data available for combination: half-ppr, 10, 2014
No data available for combination: half-ppr, 10, 2015
No data available for combination: half-ppr, 10, 2016
No data available for combination: half-ppr, 10, 2017
No data available for combination: half-ppr, 12, 2007
No data available for combination: half-ppr, 12, 2008
No data available for combination: half-ppr, 12, 2009
No data available for combination: half-ppr, 12, 2010
No data available for combination: half-ppr, 12, 2011
No data available for combination: half-ppr, 12, 2012
No data available for combination: half-ppr, 12, 2013
No data available for combin

,season,teams,scoring,player_id,name,position,team,adp,adp_formatted,times_drafted,high,low,stdev,bye
0,2008,10,standard,1173.0,LaDainian Tomlinson,RB,LAC,1.4,1.01,813.0,1.0,5.0,0.7,8.0
1,2008,10,standard,925.0,Adrian Peterson,RB,MIN,2.4,1.02,417.0,1.0,6.0,0.7,7.0
2,2008,10,standard,1237.0,Brian Westbrook,RB,PHI,2.8,1.03,405.0,1.0,9.0,0.7,7.0
3,2008,10,standard,4.0,Joseph Addai,RB,IND,4.7,1.05,559.0,1.0,8.0,1.1,14.0
4,2008,10,standard,581.0,Steven Jackson,RB,LAR,5.2,1.05,658.0,1.0,11.0,1.4,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,2021,12,ppr,2987.0,Marquez Valdes-Scantling,WR,GB,170.9,15.03,47.0,109.0,209.0,23.5,14.0
207,2021,12,ppr,2129.0,Carlos Hyde,RB,JAX,172.7,15.05,60.0,134.0,207.0,14.7,11.0
208,2021,12,ppr,1315.0,Kansas City Defense,DEF,KC,173.2,15.05,43.0,120.0,209.0,25.7,8.0
209,2021,12,ppr,2988.0,Daniel Carlson,PK,LV,173.7,15.06,47.0,126.0,206.0,18.0,6.0


In [43]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11644 entries, 0 to 210
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   season         11644 non-null  int64  
 1   teams          11644 non-null  int64  
 2   scoring        11644 non-null  object 
 3   player_id      11644 non-null  float64
 4   name           11644 non-null  object 
 5   position       11644 non-null  object 
 6   team           11546 non-null  object 
 7   adp            11644 non-null  float64
 8   adp_formatted  11644 non-null  object 
 9   times_drafted  11644 non-null  float64
 10  high           11644 non-null  float64
 11  low            11644 non-null  float64
 12  stdev          11644 non-null  float64
 13  bye            11546 non-null  float64
dtypes: float64(7), int64(2), object(5)
memory usage: 1.3+ MB


## Cleaning

In [69]:
df_all['round'] = df_all['adp_formatted'].apply(lambda x: int(x.split('.')[0]))
df_all['pick'] = df_all['adp_formatted'].apply(lambda x: int(x.split('.')[1]))

df_all.head(10)

,season,teams,scoring,player_id,name,position,team,adp,adp_formatted,times_drafted,high,low,stdev,bye,round,pick
0,2008,10,standard,1173.0,LaDainian Tomlinson,RB,LAC,1.4,1.01,813.0,1.0,5.0,0.7,8.0,1,1
1,2008,10,standard,925.0,Adrian Peterson,RB,MIN,2.4,1.02,417.0,1.0,6.0,0.7,7.0,1,2
2,2008,10,standard,1237.0,Brian Westbrook,RB,PHI,2.8,1.03,405.0,1.0,9.0,0.7,7.0,1,3
3,2008,10,standard,4.0,Joseph Addai,RB,IND,4.7,1.05,559.0,1.0,8.0,1.1,14.0,1,5
4,2008,10,standard,581.0,Steven Jackson,RB,LAR,5.2,1.05,658.0,1.0,11.0,1.4,7.0,1,5
5,2008,10,standard,119.0,Tom Brady,QB,NE,5.7,1.06,608.0,1.0,14.0,2.3,10.0,1,6
6,2008,10,standard,52.0,Marion Barber,RB,DAL,6.8,1.07,785.0,2.0,14.0,1.8,9.0,1,7
7,2008,10,standard,854.0,Randy Moss,WR,NE,8.4,1.08,668.0,3.0,16.0,2.0,10.0,1,8
8,2008,10,standard,437.0,Frank Gore,RB,SF,9.3,1.09,596.0,3.0,17.0,2.1,9.0,1,9
9,2008,10,standard,948.0,Clinton Portis,RB,WAS,10.2,1.10,592.0,4.0,18.0,2.3,14.0,1,10


## Load Table to Database

In [70]:
# Save to database
df_all.to_sql('adp', conn, index=False, if_exists='replace')

11644

## Analysis

For fun, let's look at the players who have the lowest average for `round`. In other words, over all the seasons, scoring formats, and league sizes in the data, what's the average round they are picked in.

In [67]:
df_all.groupby('name')['round'].mean().sort_values()[:10]

name
Ezekiel Elliott          1.000000
Saquon Barkley           1.041667
Christian McCaffrey      1.464286
Clyde Edwards-Helaire    1.500000
Calvin Johnson           1.607143
Dalvin Cook              1.678571
Najee Harris             1.833333
LeVeon Bell              2.000000
Davante Adams            2.000000
Michael Turner           2.214286
Name: round, dtype: float64

A little surprising to see Ezekiel Elliott as the overall number 1 here. He is a workhorse back but he hasn't been that elite in recent years.

In [73]:
df_all.query("name == 'Ezekiel Elliott'")

,season,teams,scoring,player_id,name,position,team,adp,adp_formatted,times_drafted,high,low,stdev,bye,round,pick
5,2016,10,standard,2343.0,Ezekiel Elliott,RB,DAL,6.3,1.06,168.0,1.0,11.0,2.1,9.0,1,6
9,2017,10,standard,2343.0,Ezekiel Elliott,RB,DAL,10.1,1.10,499.0,1.0,20.0,4.2,9.0,1,10
3,2018,10,standard,2343.0,Ezekiel Elliott,RB,DAL,3.7,1.04,629.0,1.0,8.0,1.3,9.0,1,4
3,2019,10,standard,2343.0,Ezekiel Elliott,RB,DAL,3.8,1.04,67.0,1.0,6.0,1.0,9.0,1,4
2,2020,10,standard,2343.0,Ezekiel Elliott,RB,DAL,3.3,1.03,447.0,1.0,8.0,1.2,9.0,1,3
4,2021,10,standard,2343.0,Ezekiel Elliott,RB,DAL,5.4,1.05,629.0,2.0,10.0,1.1,9.0,1,5
5,2016,12,standard,2343.0,Ezekiel Elliott,RB,DAL,6.3,1.06,168.0,1.0,11.0,2.1,9.0,1,6
9,2017,12,standard,2343.0,Ezekiel Elliott,RB,DAL,10.1,1.10,499.0,1.0,20.0,4.2,9.0,1,10
3,2018,12,standard,2343.0,Ezekiel Elliott,RB,DAL,3.7,1.04,629.0,1.0,8.0,1.3,9.0,1,4
3,2019,12,standard,2343.0,Ezekiel Elliott,RB,DAL,3.8,1.04,67.0,1.0,6.0,1.0,9.0,1,4


The data shows, he is always picked in the first round – at least in this dataset.